In [1]:
import torch
import h5py
import yaml
import os

from torch_geometric.utils import to_undirected, add_self_loops

from laplacian import *

In [2]:
import sys
import importlib.util

solver_dir = "/home/ubuntu/SML/solver"

sys.path.append(solver_dir)

file_path = "/home/ubuntu/SML/solver/solver.py"
spec = importlib.util.spec_from_file_location("solver", file_path)
solver = importlib.util.module_from_spec(spec)
spec.loader.exec_module(solver)

In [3]:
E = 113.8e9
nu = 0.342
rho = 4.47e-3

In [4]:
device = "cuda:2"
dtype = torch.float32

In [5]:
files = os.listdir("/data/SimJEB/small_ver_25/preprocessed/")
print(files)

['sample_19.h5', 'sample_35.h5', 'sample_40.h5', 'sample_0.h5', 'sample_27.h5', 'sample_14.h5', 'sample_38.h5', 'sample_10.h5', 'sample_12.h5', 'sample_29.h5', 'sample_28.h5', 'sample_23.h5', 'sample_8.h5', 'sample_63.h5', 'sample_4.h5', 'sample_16.h5', 'sample_6.h5', 'sample_21.h5', 'sample_20.h5', 'sample_15.h5', 'sample_30.h5', 'sample_33.h5', 'sample_62.h5', 'sample_22.h5', 'sample_9.h5']


In [6]:
i = 18

idx = files[i].split('_')[1].split('.')[0]
print(idx)

with open(f'/data/SimJEB/boundary/{idx}.yaml') as file:
    boundary = yaml.safe_load(file)

with h5py.File(f"/data/SimJEB/small_ver_25/preprocessed/"+files[i], 'r') as f:
    coords = torch.tensor(f['coords'][:])
    elements = torch.tensor(f['elements'][:])
    displacement = torch.tensor(f['outputs'][:])[:,0,:3]
    stress = torch.tensor(f['outputs'][:])[:,0,-1] * 1e6   # scale unit (MPa to Pa)

coords = coords.to(device=device, dtype=dtype)
elements = elements.to(device=device, dtype=torch.int32)
displacement = displacement.to(device=device, dtype=dtype)
stress = stress.to(device=device, dtype=dtype)

20


In [ ]:
for i in range(len(files)):
    print(i,end="\r")
    idx = files[i].split('_')[1].split('.')[0]

    with open(f'/data/SimJEB/boundary/{idx}.yaml') as file:
        boundary = yaml.safe_load(file)

    with h5py.File(f"/data/SimJEB/small_ver_25/preprocessed/sample_{idx}.h5", 'r') as f:
        coords = torch.tensor(f['coords'][:])
        elements = torch.tensor(f['elements'][:])
        displacement = torch.tensor(f['outputs'][:])[:,0,:3]
        stress = torch.tensor(f['outputs'][:])[:,0,-1]

    coords = coords.to(device=device, dtype=dtype)
    elements = elements.to(device=device, dtype=torch.int32)
    displacement = displacement.to(device=device, dtype=dtype)
    stress = stress.to(device=device, dtype=dtype)

    edge, tetra_edge_id, tetra_edge_orientation = tetrahedral_to_full_edge_data(elements, device)
    coord_gradient = compute_target_gradient(coords, edge) # [E, 3]

    displacement_gradient = compute_target_gradient(displacement, edge) # [E, 3]

    edge_edge = compute_edge_edge(tetra_edge_id, device) # [E', 2]

    rbe2_list = [rbe2['slaves'] for rbe2 in boundary['rbe2']]
    rbe3_list = [rbe3['slaves'] for rbe3 in boundary['rbe3']]
    edge_rbe2_mask_list = []
    edge_rbe3_mask_list = []
    for rbe2 in rbe2_list:
        rbe2_ids = torch.tensor(rbe2, device=device)
        mask = torch.isin(edge[:, 0], rbe2_ids) & torch.isin(edge[:, 1], rbe2_ids)
        edge_rbe2_mask = torch.zeros(edge.shape[0], dtype=torch.bool, device=device)
        edge_rbe2_mask |= mask
        edge_rbe2_mask_list.append(mask)
    for rbe3 in rbe3_list:
        rbe3_ids = torch.tensor(rbe3, device=device)
        mask = torch.isin(edge[:, 0], rbe3_ids) & torch.isin(edge[:, 1], rbe3_ids)
        edge_rbe3_mask = torch.zeros(edge.shape[0], dtype=torch.bool, device=device)
        edge_rbe3_mask |= mask
        edge_rbe3_mask_list.append(mask)

    edge_length = torch.norm(coord_gradient, dim=-1) # [E]
    rbe2_hop_length = []
    for i in range(len(edge_rbe2_mask_list)):
        hop_length = compute_length_sum_for_edge(edge_edge, edge.shape[0], edge_rbe2_mask_list[i], edge_length, device)
        rbe2_hop_length.append(hop_length)
    rbe3_hop_length = []
    for i in range(len(edge_rbe3_mask_list)):
        hop_length = compute_length_sum_for_edge(edge_edge, edge.shape[0], edge_rbe3_mask_list[i], edge_length, device)
        rbe3_hop_length.append(hop_length)
    rbe2_hop_length = torch.exp(torch.log(torch.stack(rbe2_hop_length, dim=0)).mean(dim=0)) # [E]
    rbe3_hop_length = torch.exp(torch.log(torch.stack(rbe3_hop_length, dim=0)).mean(dim=0)) # [E]

    x = torch.cat([coord_gradient, edge_length.unsqueeze(-1)], dim=-1) # [E, 4]
    bc = torch.cat([rbe2_hop_length.unsqueeze(-1), rbe3_hop_length.unsqueeze(-1)], dim=-1) # [E, 2]
    edge_index = add_self_loops(edge_edge.transpose(0, 1))[0] # [2, E']
    edge_weight = torch.ones(edge_index.shape[1], device=device) # [E']
    y = displacement_gradient # [E, 3]

    element_vm_stress = solver.compute_element_stress(coords, elements, displacement, E, nu, "c3d4", device)[1] / 1e6

    with h5py.File(f"/data/SimJEB/laplacian/{idx}.h5", 'w') as f:
        x = f.create_dataset('x', data=x.cpu().numpy())
        bc = f.create_dataset('bc', data=bc.cpu().numpy())
        edge_index = f.create_dataset('edge_index', data=edge_index.cpu().numpy())
        edge_weight = f.create_dataset('edge_weight', data=edge_weight.cpu().numpy())
        y = f.create_dataset('y', data=y.cpu().numpy())
        stress = f.create_dataset('stress', data=stress.cpu().numpy())
        tetra_edge_id = f.create_dataset('tetra_edge_id', data=tetra_edge_id.cpu().numpy())
        tetra_edge_orientation = f.create_dataset('tetra_edge_orientation', data=tetra_edge_orientation.cpu().numpy())
        coords = f.create_dataset('coords', data=coords.cpu().numpy())
        elements = f.create_dataset('elements', data=elements.cpu().numpy())
        edge = f.create_dataset('edge', data=edge.cpu().numpy())
        displacement = f.create_dataset('displacement', data=displacement.cpu().numpy())
        element_vm_stress = f.create_dataset('element_vm_stress', data=element_vm_stress.cpu().numpy())

tensor([58.8687, 85.8151, 91.1977, 67.9958, 18.9751, 38.8002, 55.0404, 46.4024,
        73.8338, 66.8318], device='cuda:0')
tensor([262.5918, 360.7728, 289.8835, 450.8071, 211.8481, 252.5427, 428.1967,
        474.8176, 363.8794, 215.3476], device='cuda:0')
tensor([135.8052, 127.3035, 143.2776, 188.1649, 152.5815,  47.7730, 234.9525,
        316.1298,  69.1931, 238.2103], device='cuda:0')


KeyboardInterrupt: 